In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"   
os.environ["CUDA_VISIBLE_DEVICES"]="0"

In [3]:
from transformers import ReformerConfig, ReformerModel
import torch
from torch import nn, Tensor

/home/clim-lab/miniconda3/envs/MEaI_transformer/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2022-11-28 18:21:08.316984: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-11-28 18:21:08.480098: E tensorflow/stream_executor/cuda/cuda_blas.cc:2981] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2022-11-28 18:21:08.946637: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7:

In [4]:
WINDOW_SIZE = 30

reformer_config = ReformerConfig(vocab_size=WINDOW_SIZE)
reformer_full = ReformerModel(reformer_config)

for i,layer in enumerate(reformer_full.children()):
    if i == 0:
        reformer_embedding = layer
    else:
        reformer_encoder = layer
reformer_encoder

ReformerEncoder(
  (layers): ModuleList(
    (0): ReformerLayer(
      (attention): ReformerAttention(
        (layer_norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (self_attention): LocalSelfAttention(
          (query): Linear(in_features=256, out_features=768, bias=False)
          (key): Linear(in_features=256, out_features=768, bias=False)
          (value): Linear(in_features=256, out_features=768, bias=False)
        )
        (output): ReformerSelfOutput(
          (dense): Linear(in_features=768, out_features=256, bias=False)
        )
      )
      (feed_forward): ChunkReformerFeedForward(
        (layer_norm): LayerNorm((256,), eps=1e-12, elementwise_affine=True)
        (dense): ReformerFeedForwardDense(
          (act_fn): ReLU()
          (dense): Linear(in_features=256, out_features=512, bias=True)
        )
        (output): ReformerFeedForwardOutput(
          (dense): Linear(in_features=512, out_features=256, bias=True)
        )
      )
    )
  

In [5]:
reformer_config

ReformerConfig {
  "attention_head_size": 64,
  "attn_layers": [
    "local",
    "lsh",
    "local",
    "lsh",
    "local",
    "lsh"
  ],
  "axial_norm_std": 1.0,
  "axial_pos_embds": true,
  "axial_pos_embds_dim": [
    64,
    192
  ],
  "axial_pos_shape": [
    64,
    64
  ],
  "chunk_size_lm_head": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "feed_forward_size": 512,
  "hash_seed": null,
  "hidden_act": "relu",
  "hidden_dropout_prob": 0.05,
  "hidden_size": 256,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-12,
  "local_attention_probs_dropout_prob": 0.05,
  "local_attn_chunk_length": 64,
  "local_num_chunks_after": 0,
  "local_num_chunks_before": 1,
  "lsh_attention_probs_dropout_prob": 0.0,
  "lsh_attn_chunk_length": 64,
  "lsh_num_chunks_after": 0,
  "lsh_num_chunks_before": 1,
  "max_position_embeddings": 4096,
  "model_type": "reformer",
  "num_attention_heads": 12,
  "num_buckets": null,
  "num_hashes": 1,
  "num_hidden_layers": 6,
  "pad_token_id": 0,


In [6]:
class compare_former_model(nn.Module):
    def __init__(self,encoder,encoder_config):
        super(compare_former_model, self).__init__()
        self.d_model = encoder_config.hidden_size
        self.encoder = encoder
        self.decoder = nn.Linear(self.d_model,1)

    def forward(self, x):
        encoder_out = self.encoder(x)
        logits = self.decoder(encoder_out.last_hidden_state)
        return logits

In [8]:
tmp = np.random.rand(30*256).reshape([1,30,256])
tmp_input = torch.tensor(tmp)

reformer_encoder(tmp_input)

TypeError: 'NoneType' object is not iterable

In [ ]:
torch.Tensor([[1,2,3,45,6]])